# Calculate reproduction rate (R0) for aerosol transmission of infectious disease within a micro-environment
This model calculates the reproduction rate (R0) for transmission of infectious diseases (Covid19) within a micro-environment. It assumes the prime method of transmission is an aerosol and that the aerosol is well mixed across the space, i.e. there are no local concentrations which may impact on an individuals probability for being infected.

The modelling follows the approach set out in the paper:
Buonanno, G., Stabile, L., & Morawska, L. (2020). Estimation of airborne viral emission: Quanta emission rate of SARS-CoV-2 for infection risk assessment [Preprint]. Infectious Diseases (except HIV/AIDS). https://doi.org/10.1101/2020.04.12.20062828

In [142]:
import math
import plotly.graph_objects as go

In [143]:
# Check parameters to ensure we don't have out of bound values
def isGreaterThanZero(x):
    if not(type(x) is int or type(x) is float):
        raise ValueError('value must be a number')
    if not (x > 0):
        raise ValueError('value must be greater than zero')

def isGreaterThanOrEqualZero(x):
    if not(type(x) is int or type(x) is float):
        raise ValueError('value must be a number')
    if not (x >= 0):
        raise ValueError('value must be greater than, or equal to, zero')

In [144]:
# Calculate the quanta concentration at time t
def quanta_concentration(t, volume, air_exchange_rate, infected, quanta_emission_rate, initial_concentration=0.0, time_interval=1.0):
    """ Calculate the quanta concentration at time t

    Keyword arguments:
    t                       Number of time intervals in the future when concentration is calculated
    volume                  Volume of the indoor environment
    air_exchange_rate       Rate at which air is exchanged
    infected                The number of infectious subjects 
    quanta_emission_rate    Rate at which an infected person emits infectious droplets
    initial_concentration   Quanta concentration at the start of the period (default=0.0)
                            Note: Original paper n0 is absolute quanta, not concentration
    time_interval           Scaling factor for time interval to enable sub-unit calculations (default=1.0)

    Return value:
    quanta_concentration    Concentration of quanta at time period t.

    Note conventions:
        Time period is measured in hours
        Volumes are metres^3
        Concentration is quanta per m^3 (for initial concentration and returned results)
        Emmission rate is quanta per hour
    """
    isGreaterThanZero(t)
    isGreaterThanZero(volume)
    isGreaterThanZero(air_exchange_rate)
    isGreaterThanOrEqualZero(infected)
    isGreaterThanZero(quanta_emission_rate)
    isGreaterThanOrEqualZero(initial_concentration)
    isGreaterThanZero(time_interval)  

    a = quanta_emission_rate * infected / (air_exchange_rate * volume) * time_interval
    quanta_concentration = a + (initial_concentration + a) * math.exp(-air_exchange_rate * t * time_interval)
    
    return quanta_concentration                    

In [145]:
# Calculate the infection risk as a function of exposure time using the Wells-Riley equation
def infection_risk(exposure_time, inhalation_rate):
    """ Calculate the infection risk based upon exposure time and inhalation rate

    Keywork arguments
    exposure_time           Amount of time that the susceptible person was exposed
    inhalation_rate         Inhalation rate of the exposed subject (affected by subjects activity levels)

    Return
    infection_risk          Risk that a susceptible person will be infected as a probability
    """

    # Assume parameters are valid (all parameters greater than zero)

    # Integrate quanta_concentration over time from 0 to T [exposure_time]
    

In [146]:
# Plot concentration in micro-environment over a period

# Set base statistics
me_volume = 75 # m^3
me_air_exchange_rate = 0.2 # h^-1: natural ventilation (0.2) mechanical ventilation (2.2)
me_quanta_emission_rate = 15.7 # quanta h^-1: Resting(98.1), Standing (147), Light Exercise (317)
me_n0 = 0.0

me_infected = 1

periods = 100
time_interval = 1/60 # minutes
n = [] # create a list for results
time_axis =[]

time_axis.append(0)
n.append(me_n0)

for t in range(1, periods):
    time_axis.append(t)
    this_n =  me_n0 if not n else n[-1] # initialise n, otherwise use last time period
    this_infected = me_infected if t <10 else 0
    n.append(quanta_concentration(1, me_volume, me_air_exchange_rate, this_infected, me_quanta_emission_rate, this_n, time_interval))

fig = go.Figure()
fig.add_trace(go.Scatter(x=time_axis, y=n , name='quanta concentration', line=dict(color='firebrick', width=1)))
fig.update_layout(title='Quanta concentration',
                   xaxis_title='Time (minutes)',
                   yaxis_title='Quanta concentration (quanta per m^-3)')
fig.show()

ValueError: value must be a number